# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlite3
import pandas as pd

In [2]:
# load data from database
df = pd.read_sql_table('Response_table', 'sqlite:///Disaster_Response.db')  
X = df.message.values
Y = df.iloc[:, 4:]

In [3]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [4]:
Y.isnull().sum()

related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

In [5]:
Y.shape

(26042, 36)

### 2. Write a tokenization function to process your text data

In [6]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [7]:
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [8]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
    

In [9]:
for message in X[:2]:
    tokens = tokenize(message)
    print(message)
    print(tokens,'\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', '-', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

pipeline = Pipeline([('vect',CountVectorizer()),('tfidf',TfidfTransformer()),('clf',MultiOutputClassifier(RandomForestClassifier()))])
    

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
#Split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.33, random_state=42)
#Train pipeline
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
y_pred = pipeline.predict(X_test)
from sklearn.metrics import classification_report
m = Y.columns
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=m))

                        precision    recall  f1-score   support

               related       0.37      0.05      0.09      1495
               request       0.00      0.00      0.00        46
                 offer       0.43      0.18      0.25      3563
           aid_related       0.11      0.01      0.01       702
          medical_help       0.03      0.00      0.00       425
      medical_products       0.00      0.00      0.00       245
     search_and_rescue       0.00      0.00      0.00       160
              security       0.00      0.00      0.00       272
              military       0.00      0.00      0.00         0
           child_alone       0.16      0.01      0.01       555
                 water       0.22      0.01      0.02       989
                  food       0.16      0.01      0.02       752
               shelter       0.00      0.00      0.00       140
              clothing       0.00      0.00      0.00       208
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 35, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [17]:
import numpy as np
from sklearn.metrics import accuracy_score
target_names = Y.columns
y_true = np.array(y_test)
accuracy = pd.DataFrame()
for i,target in enumerate(target_names):
    Accuracy_Score = accuracy_score(y_true[:, i], y_pred[:, i])
    accuracy = accuracy.append({'Accuracy':Accuracy_Score,'index':target},ignore_index = True)
print(accuracy)

    Accuracy                   index
0   0.729579                 related
1   0.819525                 request
2   0.994415                   offer
3   0.562253             aid_related
4   0.914242            medical_help
5   0.946940        medical_products
6   0.970212       search_and_rescue
7   0.980684                security
8   0.967885                military
9   1.000000             child_alone
10  0.933442                   water
11  0.881894                    food
12  0.909006                 shelter
13  0.983593                clothing
14  0.975448                   money
15  0.988597          missing_people
16  0.964277                refugees
17  0.952990                   death
18  0.863859               other_aid
19  0.936235  infrastructure_related
20  0.952990               transport
21  0.945194               buildings
22  0.979055             electricity
23  0.993600                   tools
24  0.988946               hospitals
25  0.994880                   shops
2

### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
#View the parameters in the pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_s

In [19]:
from sklearn.model_selection import GridSearchCV
parameters = {'tfidf__use_idf':[True, False],
              'clf__estimator__min_samples_leaf': [1, 2],
              'clf__estimator__min_samples_split': [2 , 3]
              }

cv = GridSearchCV(pipeline, param_grid = parameters )

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
model_tunned = cv.fit(X_train,y_train)
y_pred = model_tunned.predict(X_test)
from sklearn.metrics import classification_report
m = Y.columns
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=m))

In [20]:
target_names = Y.columns
y_true = np.array(y_test)
accuracy = pd.DataFrame()
for i,target in enumerate(target_names):
    Accuracy_Score = accuracy_score(y_true[:, i], y_pred[:, i])
    accuracy = accuracy.append({'Accuracy':Accuracy_Score,'index':target},ignore_index = True)
print(accuracy)

    Accuracy                   index
0   0.732721                 related
1   0.820922                 request
2   0.994647                   offer
3   0.570980             aid_related
4   0.915988            medical_help
5   0.948103        medical_products
6   0.970910       search_and_rescue
7   0.981266                security
8   0.968001                military
9   1.000000             child_alone
10  0.933209                   water
11  0.881545                    food
12  0.909821                 shelter
13  0.983477                clothing
14  0.975797                   money
15  0.988597          missing_people
16  0.964626                refugees
17  0.952758                   death
18  0.864208               other_aid
19  0.935653  infrastructure_related
20  0.953805               transport
21  0.945078               buildings
22  0.978590             electricity
23  0.993600                   tools
24  0.988713               hospitals
25  0.994880                   shops
2

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
from sklearn.neighbors import KNeighborsClassifier
pipeline = Pipeline([('vect',CountVectorizer()),('tfidf',TfidfTransformer()),('clf',MultiOutputClassifier(KNeighborsClassifier()))])
    

In [19]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
              metric_params=None, n_jobs=1, n_neighbors=5, p=2,
              weights='uniform'),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ng

In [20]:
parameters= {'tfidf__use_idf':[True, False],
             'clf__estimator__n_neighbors': [5,10]
            }
cv = GridSearchCV(pipeline, param_grid = parameters )

In [ ]:
#Test the model again
Model_KN = cv.fit(X_train, y_train)

In [ ]:
y_pred = Model_KN.predict(X_test)

In [ ]:
m = Y.columns
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), target_names=m))

In [ ]:
#Test the accuracy:
target_names = Y.columns
y_true = np.array(y_test)
accuracy = pd.DataFrame()
for i,target in enumerate(target_names):
    Accuracy_Score = accuracy_score(y_true[:, i], y_pred[:, i])
    accuracy = accuracy.append({'Accuracy':Accuracy_Score,'index':target},ignore_index = True)
print(accuracy)

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(model_tunned, open('classifier.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.